## Sending Emails With Python

In [ ]:
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from email.mime.text import MIMEText
import base64
import os.path
import pickle

from datetime import datetime

# If modifying these scopes, delete the file token.pickle.
SCOPES = ['https://www.googleapis.com/auth/gmail.send']

def get_credentials():
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)
    
    return creds

def send_email(service, to, subject, body, from_email):

    time = datetime.now().isoformat()

    message = MIMEText(body + f'\n\nSent at {time}')
    message['to'] = to
    message['subject'] = subject
    message['from'] = from_email

    raw_message = base64.urlsafe_b64encode(message.as_bytes()).decode('utf-8')

    try:
        message = service.users().messages().send(userId='me', body={'raw': raw_message}).execute()
        print(f'Message Id: {message["id"]}')
        print(f'Email sent successfully to {to} from {from_email} at {time}')
    except Exception as error:
        print(f'An error occurred: {error}')

def main():
    creds = get_credentials()
    service = build('gmail', 'v1', credentials=creds)
    
    send_email(
        service,
        to='mlevydaniel@gmail.com',
        subject='Test Email from Gmail API sent',
        body=f'This is a test email sent using the Gmail API.',
        from_email='mlevydanieltest@gmail.com'
    )

if __name__ == '__main__':
    main()

In [ ]:
main()

In [ ]:
import base64
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from google.oauth2.credentials import Credentials
from google.auth.transport.requests import Request
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import os.path
import pickle

from datetime import datetime

# If modifying these scopes, delete the file token.pickle.
SCOPES = ['https://www.googleapis.com/auth/gmail.send']

def get_credentials():
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)
    return creds

def create_message(sender, to, subject, html):
    message = MIMEMultipart("alternative")
    message["Subject"] = subject
    message["From"] = sender
    message["To"] = to

    part = MIMEText(html, "html")

    message.attach(part)

    raw_message = base64.urlsafe_b64encode(message.as_bytes()).decode("utf-8")
    return {'raw': raw_message}

def send_message(service, user_id, message):
    try:
        message = service.users().messages().send(userId=user_id, body=message).execute()
        print(f'Message Id: {message["id"]}')
        print('Email sent successfully')
        return message
    except HttpError as error:
        print(f'An error occurred: {error}')

def main():
    creds = get_credentials()
    service = build('gmail', 'v1', credentials=creds)

    time = datetime.now().isoformat()
    sender_email = "mlevydanieltest@gmail.com"  # Replace with your Gmail address
    receiver_email = "mlevydaniel@gmail.com"  # Replace with recipient's email
    subject = "Gmail API test with OAuth"
    
    html = f"""\
    <html>
      <body>
        <p>Hi,<br>
           How are you?<br>
           This is a <b>test email</b> sent using <i>Gmail API with OAuth authentication at {time}</i>.
        </p>
      </body>
    </html>
    """

    message = create_message(sender_email, receiver_email, subject, html)
    send_message(service, "me", message)

if __name__ == '__main__':
    main()

### Using SMTP Server

In [ ]:
import base64
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from google.oauth2.credentials import Credentials
from google.auth.transport.requests import Request
from google_auth_oauthlib.flow import InstalledAppFlow
import os.path
import pickle

SCOPES = ['https://mail.google.com/']


def get_credentials():
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)
    return creds

def create_message(sender, to, subject, html):
    message = MIMEMultipart("alternative")
    message["Subject"] = subject
    message["From"] = sender
    message["To"] = to

    part2 = MIMEText(html, "html")

    message.attach(part2)

    return message

def send_message_smtp(creds, sender, to, message):
    try:
        access_token = creds.token
        auth_string = f'user={sender}\1auth=Bearer {access_token}\1\1'
        auth_bytes = auth_string.encode('ascii')
        auth_b64 = base64.b64encode(auth_bytes)

        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.starttls()
        # server.ehlo()
        # server.docmd('AUTH', 'XOAUTH2 ' + auth_b64.decode('ascii'))
        # server.ehlo()
        server.login(sender, access_token)  # Use login with sender email and access token
        server.sendmail(sender, to, message.as_string())
        server.quit()
        print('Email sent successfully via SMTP')
    except smtplib.SMTPAuthenticationError as e:
        print(f'SMTP Authentication failed: {e}')
    except Exception as e:
        print(f'An error occurred: {e}')

def main():
    creds = get_credentials()

    sender_email = "mlevydanieltest@gmail.com"  # Replace with your Gmail address
    receiver_email = "mlevydaniel@gmail.com"  # Replace with recipient's email
    subject = "SMTP test with OAuth"

    html = """\
    <html>
      <body>
        <p>Hi,<br>
           How are you?<br>
           This is a <b>test email</b> sent using <i>SMTP with OAuth authentication</i>.
        </p>
      </body>
    </html>
    """

    message = create_message(sender_email, receiver_email, subject, html)
    send_message_smtp(creds, sender_email, receiver_email, message)

if __name__ == '__main__':
    main()

### Telegram

In [1]:
import logging
import os
from dotenv import load_dotenv
from telegram import Bot
from telegram.error import TelegramError

# Load environment variables from .env so we don't hardcode secrets in the notebook
load_dotenv()

BOT_TOKEN = os.getenv("TELEGRAM_BOT_TOKEN")
CHAT_ID = os.getenv("TELEGRAM_CHAT_ID")


class TelegramMessenger:
    def __init__(self, bot_token, log_level=logging.INFO):
        self.bot_token = bot_token
        self.bot = Bot(token=self.bot_token)
        self.logger = self._setup_logger(log_level)

    def _setup_logger(self, log_level):
        logger = logging.getLogger(__name__)
        logger.setLevel(log_level)
        handler = logging.StreamHandler()
        formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
        handler.setFormatter(formatter)
        logger.addHandler(handler)
        return logger

    def send_message(self, chat_id, message):
        try:
            self.logger.info(f"Sending message to chat_id: {chat_id}")
            self.bot.send_message(chat_id=chat_id, text=message)
            self.logger.info("Message sent successfully")
        except TelegramError as e:
            self.logger.error(f"Failed to send message: {str(e)}")
            raise

    def send_photo(self, chat_id, photo_path, caption=None):
        try:
            self.logger.info(f"Sending photo to chat_id: {chat_id}")
            with open(photo_path, 'rb') as photo:
                self.bot.send_photo(chat_id=chat_id, photo=photo, caption=caption)
            self.logger.info("Photo sent successfully")
        except TelegramError as e:
            self.logger.error(f"Failed to send photo: {str(e)}")
            raise
        except IOError as e:
            self.logger.error(f"Failed to open photo file: {str(e)}")
            raise


if __name__ == "__main__":
    if not BOT_TOKEN or not CHAT_ID:
        raise RuntimeError("TELEGRAM_BOT_TOKEN and TELEGRAM_CHAT_ID must be set in the .env file")

    messenger = TelegramMessenger(BOT_TOKEN)

    try:
        messenger.send_message(CHAT_ID, "Hello, Telegram!")
        # Uncomment the next line if you have a photo to send
        # messenger.send_photo(CHAT_ID, "path/to/photo.jpg", "Check out this photo!")
    except Exception as e:
        print(f"An error occurred: {str(e)}")

2024-07-16 11:36:27,508 - __main__ - INFO - Sending message to chat_id: 693527469
/var/folders/s5/rdg9kttj7n9f1wwq0lrqtz_c0000gn/T/ipykernel_4862/1137169836.py:23: RuntimeWarning: coroutine 'Bot.send_message' was never awaited
  self.bot.send_message(chat_id=chat_id, text=message)
2024-07-16 11:36:27,509 - __main__ - INFO - Message sent successfully
